In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import *
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
#extract development and evaulation
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [ ]:
df.describe

In [ ]:
condition = (df['Current language used for work/school'] == np.unique(df_eval['Current language used for work/school'].values)[0]) & (df['First Language spoken'] == np.unique(df_eval['First Language spoken'].values)[0]) & (df['Self-reported fluency level '] == np.unique(df_eval['Self-reported fluency level '].values)[0])
df = df.loc[condition]



In [ ]:
df.describe

In [ ]:
np.unique(df_eval['Self-reported fluency level '].values)

In [ ]:
df['intent'] = df['action']+ df['object']
# encoding y
dict = {}
for i, el in enumerate(df['intent'].unique()):
    dict[el] = i

df['intent'] = df['intent'].apply(lambda x: dict[x])

In [ ]:
#dropping irrelevant columns
df.drop(columns=["Self-reported fluency level ","First Language spoken", "Current language used for work/school","action","object", "intent"], inplace=True)


In [ ]:
df_eval.drop(columns=["Self-reported fluency level ","First Language spoken", "Current language used for work/school"], inplace=True)

In [ ]:
# encode gender 
label_encoder(df, 'gender')
label_encoder(df_eval, 'gender')

In [ ]:
label_encoder(df, 'ageRange')
label_encoder(df_eval, 'ageRange')

In [ ]:
audio_feature_extraction(df)
audio_feature_extraction(df_eval)

In [ ]:
chroma_feature(df)
chroma_feature(df_eval)
time_domain_2D(df,'chroma')
time_domain_2D(df_eval,'chroma')

arr = ['chroma_mean','chroma_min','chroma_max','chroma_std','chroma_kurtosis','chroma_skew']
for i in arr:
    array_column_spread(df,i)
    array_column_spread(df_eval,i)

In [ ]:
tonnetz_feature(df)
tonnetz_feature(df_eval)

time_domain_2D(df,'tonnetz')
time_domain_2D(df_eval,'tonnetz')

arr = ['tonnetz_mean','tonnetz_min','tonnetz_max','tonnetz_std','tonnetz_kurtosis','tonnetz_skew']
for i in arr:
    array_column_spread(df,i)
    array_column_spread(df_eval,i)

In [ ]:
spectral_contrast(df)
spectral_contrast(df_eval)

time_domain_2D(df,'spectral_contrast')
time_domain_2D(df_eval,'spectral_contrast')

arr = ['spectral_contrast_mean','spectral_contrast_min','spectral_contrast_max','spectral_contrast_std','spectral_contrast_kurtosis','spectral_contrast_skew']
for i in arr:
    array_column_spread(df,i)
    array_column_spread(df_eval,i)

In [ ]:
rmse_feature(df)
rmse_feature(df_eval)

time_domain_1D(df,'rmse')
time_domain_1D(df_eval,'rmse')

In [ ]:
spectral_flatness(df)
spectral_flatness(df_eval)
time_domain_1D(df,'sf')
time_domain_1D(df_eval,'sf')

In [ ]:
sro_feature(df)
sro_feature(df_eval)
time_domain_1D(df,'spectral_rolloff')
time_domain_1D(df_eval,'spectral_rolloff')

In [ ]:
zcr_feature(df)
zcr_feature(df_eval)

In [ ]:
mfcc_feature(df)
mfcc_feature(df_eval)

time_domain_2D(df,'mfcc')
time_domain_2D(df_eval,'mfcc')

arr = ['mfcc_mean','mfcc_min','mfcc_max','mfcc_std','mfcc_kurtosis','mfcc_skew']
for i in arr:
    array_column_spread(df,i)
    array_column_spread(df_eval,i)

In [ ]:
def mfcc_feature(df):
    # Create arrays to store the mfcc rate values
    mfcc_array = []

    for data, rate in zip(df['data'], df['rate']):
        
        # Compute the mfccs for the current audio file
        mfcc = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=30)
        mfcc_array.append(mfcc)

    # Add the mfcc as a new column in the dataframe
    df['mfcc'] = mfcc_array

In [ ]:
df

In [ ]:
df2 = df.copy()
df3 = df_eval.copy()

# Iterate over each column in the DataFrame
for col in df2.columns:
    # Check if the column contains 1D or 2D arrays
    if isinstance(df2[col].values[0], (list, np.ndarray)) and len(df2[col].values[0]) > 1:
        # Drop the column if it contains 1D or 2D arrays
        df2.drop(col, axis=1, inplace=True)
for col in df3.columns:
    # Check if the column contains 1D or 2D arrays
    if isinstance(df3[col].values[0], (list, np.ndarray)) and len(df3[col].values[0]) > 1:
        # Drop the column if it contains 1D or 2D arrays
        df3.drop(col, axis=1, inplace=True)

In [ ]:
label_encoder(df2, 'ageRange')
label_encoder(df3, 'ageRange')
df2.drop(columns=['path','speakerId','rate'],inplace = True)
df3.drop(columns=['path','speakerId','rate'],inplace = True)

In [ ]:
X = df2.copy()
X_eval = df3.copy()

# X.drop(columns=['Id', 'path', 'speakerId', 'action', 'object', 'intent'], inplace=True)

X.to_csv('x.csv',index=False)

# X_eval.drop(columns=['Id', 'path', 'speakerId'], inplace=True)

X_eval.to_csv('x_eval.csv', index=False)


## Step 2

In [69]:
y = df['intent'].values
X = pd.read_csv("x.csv")
X_eval = pd.read_csv("x_eval.csv")

In [72]:
from sklearn.preprocessing import MinMaxScaler


X =  normalize_dataframe(X)
X_eval = normalize_dataframe(X_eval)

In [ ]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
import pandas as pd

# Create dataframe

# Create SVM classifier
svm = SVC(kernel="linear")

# Use RFE to select top n features
rfe = RFE(svm, n_features_to_select=n)
rfe.fit(X, y)

# Get the top n features
top_n_features = X.columns[rfe.support_]

In [ ]:
top = best_features_SVC(X,y,90)

In [ ]:
X_top = X[top]
X_eval_top = X_eval[top]

In [ ]:
#update y
y = np.delete(y, noise_df.index)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def svm_model2(X_train, y_train, X_test):
    clf = SVC(C = 100, gamma = 0.01, kernel = 'rbf')
    # train the model on the training data
    clf.fit(X_train, y_train)
    # predict the target values for the test data
    # returning the y_predict
    return clf.predict(X_test)
y_pred = svm_model2(X, y, X_eval)

In [ ]:
svm_accuracy = accuracy_calculator(y_test, y_pred)
svm_accuracy

In [ ]:
key_mapping = np.vectorize(lambda x: next(key for key, value in dict.items() if value == x))
keys_arr = key_mapping(y_pred)
pd.Series(keys_arr).to_csv('predictions.csv')

In [ ]:

np.unique(y_pred)